# Obtenção dos Dados:

Este projeto foi inspirado pelas atividades previamente realizadas por [Kizzy Terra](#https://github.com/kterra) e os dados utilizados estão neste [repositório](#https://github.com/kterra/ciencia-de-dados-aplicada), que segundo a própria autora foram dados obtidos pelo PRODES.

_"O projeto PRODES realiza o monitoramento por satélites do desmatamento por corte raso na Amazônia Legal e produz, desde 1988, as taxas anuais de desmatamento na região, que são usadas pelo governo brasileiro para o estabelecimento de políticas públicas."_ - [Coordenação Geral de Observação da Terra INPE](#http://www.obt.inpe.br/OBT/assuntos/programas/amazonia/prodes#:~:text=O%20projeto%20PRODES%20realiza%20o,o%20estabelecimento%20de%20pol%C3%ADticas%20p%C3%BAblicas.)

In [1]:
from modules import *

# DATASETS = [pd.read_csv(f'../Data/DesmatamentoMunicipios20{digits}.txt', sep=',', encoding='iso-8859-1') for digits in range(0,18)]
DATASETS = []

for i in range(0,19):
    a = '../Data/2000_2018/DesmatamentoMunicipiosAM2000.txt'
    if i < 10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM200{i}.txt', sep=',', encoding='iso-8859-1'))
    elif i>=10: DATASETS.append(pd.read_csv(f'../Data/2000_2018/DesmatamentoMunicipiosAM20{i}.txt', sep=',', encoding='iso-8859-1'))
     

type(DATASETS[0])


pandas.core.frame.DataFrame

**DATASETS** é uma lista com 19 dataframes com as informações dos arquivos que estão no diretório _`../Data/2000_2018`_

In [2]:
DATASETS[0].sample(3)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2000,Incremento19992000,Floresta2000,Nuvem2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000,Soma
52,53,-5.49767,-70.32124,S 5 29 51.61,O 70 19 16.46,Benjamin Constant,1300607,AM,9146,175.8,NaN,8748.8,0.0,16.6,0.0,204.8,100
30,31,-1.38598,-68.79404,S 1 23 9.53,O 68 47 38.54,Japurá,1302108,AM,57555,43.5,NaN,55889.4,1.0,98.3,2.6,1520.2,100
13,14,-2.50853,-59.60122,S 2 30 30.71,O 59 36 4.39,Rio Preto da Eva,1303569,AM,5838,500.6,NaN,5326.0,0.0,1.1,0.0,10.3,100


In [3]:
DATASETS[1].sample(2)

,Nr,Lat,Long,Latgms,Longms,Municipio,CodIbge,Estado,AreaKm2,Desmatado2001,Incremento20002001,Floresta2001,Nuvem2001,NaoObservado2001,NaoFloresta2001,Hidrografia2001,Soma
43,44,-3.30477,-59.55597,S 3 18 17.17,O 59 33 21.49,Careiro da Várzea,1301159,AM,2641,379.1,10.7,925.9,0.0,0.2,707.8,628.0,100
13,14,-2.50853,-59.60122,S 2 30 30.71,O 59 36 4.39,Rio Preto da Eva,1303569,AM,5838,515.6,14.9,5311.0,0.0,1.1,0.0,10.3,100


# Limpando os Dados:

Esse...

!! [...](#www.loremipsum.com)

### Removendo colunas que não são importantes:

In [4]:
print(DATASETS[0].columns)

Index(['Nr', 'Lat', 'Long', 'Latgms', 'Longms', 'Municipio', 'CodIbge',
       'Estado', 'AreaKm2', 'Desmatado2000', 'Incremento19992000',
       'Floresta2000', 'Nuvem2000', 'NaoObservado2000', 'NaoFloresta2000',
       'Hidrografia2000', 'Soma '],
      dtype='object')


In [5]:
DATASETS[0].drop(columns = ['Incremento19992000'], inplace = True)

In [6]:

DATASETS[0]

for index, value in enumerate(DATASETS):
    print(index)
    if index < 10:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem200{index}'])
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Soma '], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)
        
    else:
        value.drop(columns = ['Nr'], inplace = True)
        value.drop(columns = ['CodIbge'], inplace = True)
        value.drop(columns = [f'Nuvem20{index}'], inplace = True)
        value.drop(columns = ['Estado'], inplace = True)
        value.drop(columns = ['Latgms'], inplace = True)
        value.drop(columns = ['Longms'], inplace = True)

# DATASETS[0].drop(columns = ['Nr'], inplace = True)

DATASETS[0].sample(2)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


,Lat,Long,Municipio,AreaKm2,Desmatado2000,Floresta2000,Nuvem2000,NaoObservado2000,NaoFloresta2000,Hidrografia2000
8,-1.89625,-58.81001,São Sebastião do Uatumã,10776,151.4,9932.6,0.0,0.5,65.7,625.8
49,-8.81858,-68.00168,Boca do Acre,22595,1209.8,21200.9,0.0,1.9,58.4,124.0


In [7]:
print(DATASETS[0].columns)

Index(['Lat', 'Long', 'Municipio', 'AreaKm2', 'Desmatado2000', 'Floresta2000',
       'Nuvem2000', 'NaoObservado2000', 'NaoFloresta2000', 'Hidrografia2000'],
      dtype='object')


### Tirando acento dos caracteres

In [8]:
#Criando método para retirar os acentos das palavras dentro deste dataset:
def substituir_caracteres(columns):  
  columns = columns.replace('Á','A').replace('Â','A').replace('Ã','A').replace('À','A').replace('É','E').replace('Ê','E').replace('È','E').replace('Í','I').replace('Ì','I').replace('Î','I').replace('Ó','O').replace('Ò','O').replace('Õ','O').replace('Ô','O').replace('Ú','U').replace('Ù','U').replace('Û','U').replace('Ç','C').replace('á','a').replace('â','a').replace('ã','a').replace('à','a').replace('é','e').replace('ê','e').replace('è','e').replace('í','i').replace('ì','i').replace('î','i').replace('ó','o').replace('ò','o').replace('õ','o').replace('ô','o').replace('ú','u').replace('ù','u').replace('û','u').replace('ç','c').replace('-', ' ').replace('.','')
  return columns

for index,dataset in enumerate(DATASETS):
  DATASETS[index]['Municipio'] = DATASETS[index]['Municipio'].apply(substituir_caracteres)

DATASETS[0]['Municipio']

0     Urucurituba
1         Urucara
2          Uarini
3       Tonantins
4            Tefe
         ...     
57           Apui
58          Anori
59          Anama
60        Amatura
61       Alvaraes
Name: Municipio, Length: 62, dtype: object

### Criando coluna de fronteira entre municípios

In [9]:

#1º Percorre todos os municípios de um único DF
#2º Pergunta se todos os outros municípios fazem fronteira com ele
    #2.1º Se sim inclui estes em uma lista
    #2.2º Se não passa

#paraleloº Cria um dicionário com o nome de todos os municípios e valor vazio como correspondente

#3º Insere essa lista como valor correspondente à chave de um dicionário






#Recebe a coluna de municípios:
# def MakeAdjascencyCity(Column):
#     for index, city in enumerate(Column):
#         print(index, city)

# def MakeAdjascencyColumns(DATASETS):
#     for index, dataFrame in enumerate(DATASETS):
#         MakeAdjascencyCity(dataFrame['Municipio'])
     
        
# MakeAdjascencyCity(DATASETS[18]['Municipio'])
# MakeAdjascencyColumns(DATASETS)